In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [16]:
def options_chain(symbol):
    '''Utility Method to Get Options for Stock'''
    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    options['expirationDate'] = pd.to_datetime(
        options['expirationDate'])

    # Boolean column if the option is a CALL, to distinguish calls from puts
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)

    return options

def get_expected_move(stock, expiration_date):
    ticker = yf.Ticker(stock)
    current_price = ticker.info['regularMarketPrice']

    data = options_chain(stock)
    dates = np.sort(data['expirationDate'].unique())

    closest = data[data['expirationDate'] == expiration_date]

    #to match stock's price with the strike price
    closest['abs'] = abs(current_price - closest['strike'])
    closest = closest.sort_values('abs')
    move = (closest[closest['CALL'] == True]['lastPrice'].iloc[0] +
            closest[closest['CALL'] == False]['lastPrice'].iloc[0]) * 1.25
    return current_price, move, expiration_date
    

In [25]:
Exp_Date = "2023-01-20"
sDate = "2023-01-01"
Ticker = "QQQ"

# Get Expected Move
move = get_expected_move(Ticker, Exp_Date)

# Magnitud of the movement
upper_move = move[0] + move[1]
lower_move = move[0] - move[1]

# Extending time period into the future
b = pd.date_range(start = sDate, periods = 20)
data = yf.download(Ticker, start = sDate)

# Adding the future dates into the data dataframe
data.index.append(b)
fig = px.line(data, x=data.index, y=data['Close'], title = 'Expected Move for the Ticker: ' + Ticker)
fig.update_xaxes(range=[sDate, Exp_Date ])
fig.add_hline(y=upper_move, line = dict(color = 'red', width =1), line_dash="dot", annotation_text=upper_move,
              annotation_position="bottom right")
fig.add_hline(y=lower_move, line = dict(color = 'red', width =1), line_dash="dot", annotation_text=lower_move, 
              annotation_position="bottom right")
fig.add_vline(x=Exp_Date, line = dict(color = 'orange', width =2), line_dash="dot")
fig.update_yaxes(title_text="Stock Price")
fig.update_xaxes(title_text="Date")
fig.update_layout(height=400, width=700, 
                  showlegend=False)

fig.show()

[*********************100%***********************]  1 of 1 completed
